In [1]:
import transformers
model_name = "intfloat/e5-large-v2"  # replace with your E5-Large model name
baseline_model = transformers.AutoModel.from_pretrained(
    model_name,
    return_dict=False,
    torchscript=True,
).eval()

# Replace this with the corresponding class in ane_transformers for E5-Large
from ane_transformers.huggingface import DistilBertForSequenceClassification as OptimizedModel
optimized_model = OptimizedModel(
    baseline_model.config).eval()
optimized_model.load_state_dict(baseline_model.state_dict())

tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
tokenized = tokenizer(
    ["Sample input text to trace the model"],
    return_tensors="pt",
    max_length=128,  # token sequence length
    padding="max_length",
)

import torch
traced_optimized_model = torch.jit.trace(
    optimized_model,
    (tokenized["input_ids"], tokenized["attention_mask"])
)

import coremltools as ct
import numpy as np
ane_mlpackage_obj = ct.convert(
    traced_optimized_model,
    convert_to="mlprogram",
    inputs=[
        ct.TensorType(
                f"input_{name}",
                    shape=tensor.shape,
                    dtype=np.int32,
                ) for name, tensor in tokenized.items()
            ],
            compute_units=ct.ComputeUnit.ALL,
)
out_path = "HuggingFace_ane_transformers_E5Large_seqLen128_batchSize1.mlpackage"
ane_mlpackage_obj.save(out_path)


ImportError: cannot import name 'DistilBertForSequenceClassification' from 'ane_transformers.huggingface' (/Users/garfieldgreglim/miniconda3/lib/python3.11/site-packages/ane_transformers/huggingface/__init__.py)